In [11]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time
from webdriver_manager.chrome import ChromeDriverManager
import sqlite3
from selenium.webdriver.firefox.service import Service

In [17]:
####################
static_sleep_time = 2
static_sleep_time_short = 0.5
####################


def setting():
    conn = sqlite3.connect('jeju2.db')
    cursor = conn.cursor()

    query = "SELECT id, placeID FROM Information WHERE placeID IS NOT NULL and crawling_ver is null"
    cursor.execute(query)
    rows = cursor.fetchall()
    print(rows)

    service = Service('./geckodriver')
    driver = webdriver.Firefox(service=service)

    driver.get("https://map.naver.com/")
    return rows, driver, cursor, conn


rows, driver, cursor, conn = setting()


def get_class_text_with_sleep(class_name, driver=driver, elastic_sleep_time=2):
    try:
        return WebDriverWait(driver, elastic_sleep_time).until(
            EC.presence_of_element_located((By.CLASS_NAME, class_name))
        ).text
    except:
        return None


def get_class_text(class_name, driver=driver):
    try:
        return driver.find_element(By.CLASS_NAME, class_name).text
    except:
        return None


def print_output(key_id, name, type_, address, phone, wt):
    wt = wt.replace('\n', '')

    if len(wt) > 20:
        wt = wt[:20] + '...'

    print(f"done\t | [{key_id}] {name} ({type_}) | {address} | {phone} | {wt}")


for row in rows:
    key_id = row[0]
    placeID = row[1]
    url = f"https://pcmap.place.naver.com/restaurant/{placeID}/home"

    driver.get(url)
    time.sleep(static_sleep_time)

    try:
        name = get_class_text_with_sleep("GHAhO")
        type_ = get_class_text("lnJFt")
        address = get_class_text("LDgIH")
        phone = get_class_text("xlx7Q")

        try:
            if driver.find_elements(By.CLASS_NAME, "gKP9i"):
                driver.find_element(By.CLASS_NAME, "gKP9i").click()
                time.sleep(static_sleep_time_short)

                additional_info = []
                we_elements = driver.find_elements(By.CLASS_NAME, "i8cJw")
                hour_elements = driver.find_elements(By.CLASS_NAME, "H3ua4")

                for we_element, hour_element in zip(we_elements, hour_elements):
                    additional_info.append(
                        f"{we_element.text} | {hour_element.text}")

                wt = "\n".join(additional_info)

        except Exception as e:
            print(f"Error while handling additional info: {e}")
        cursor.execute(
            'UPDATE Information SET MCT_NAVER_NAME = ?, NAVER_ADDR = ?, MCT_TYPE = ?, PHONE = ?, WT = ?, crawling_ver = ? WHERE id = ?', (
                name, address, type_, phone, wt, 1, key_id)
        )
        conn.commit()
        print_output(key_id, name, type_, address, phone, wt)

    except Exception as e:
        print(f"err  \t | [{key_id}] {placeID} - {e}")

driver.quit()
cursor.close()
conn.close()

[(2, '1418972938'), (4, '1463424775'), (8, '1199772319'), (9, '1471178944'), (10, '1420348866'), (12, '1415013033'), (17, '13581342'), (21, '1947176580'), (22, '1529840514'), (23, '1060330773'), (26, '1789181978'), (27, '12883219'), (29, '1023103834'), (40, '1332337704'), (45, '1729610778'), (46, '32520687'), (49, '842229553'), (52, '1140636590'), (56, '1427562987'), (57, '37622421'), (59, '1695475563'), (62, '31499631'), (63, '1921928717'), (65, '1090905770'), (67, '33211465'), (68, '1560593434'), (69, '37402168'), (70, '1894018574'), (71, '1220007047'), (72, '1568141022'), (74, '1745030583'), (75, '1174429005'), (78, '1544906112'), (79, '1424589909'), (80, '1713862294'), (81, '1207379702'), (83, '13467718'), (85, '1584396762'), (86, '32155010'), (87, '1419269485'), (89, '16723803'), (90, '1028483996'), (91, '1212258834'), (92, '11730904'), (94, '1869656386'), (95, '1264153834'), (96, '1274041067'), (97, '1323898312'), (99, '1724563592'), (101, '1736374666'), (102, '1925347467'), (103

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:679:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.navigateTo@chrome://remote/content/marionette/driver.sys.mjs:859:39
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20
